# **Ch 9 - Model Inference**

In [6]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Dropout, GlobalAveragePooling1D
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from collections import Counter
from nltk.corpus import stopwords
import re
import nltk
import pickle
nltk.download('stopwords')
from nltk.corpus import stopwords
from wordcloud import WordCloud
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
# Load model
model = load_model('best_model.h5')  # or 'best_model.keras' if you used that format

# Load tokenizer
with open('tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)


In [8]:
stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower()
    text = re.sub(r"<.*?>", "", text)
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    text = re.sub(r"\bbr\b", "", text)
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)


In [9]:
# Example text
sample_review = "I really enjoyed the movie. The plot was strong and the acting was superb!"

# Clean, tokenize, and pad
cleaned = clean_text(sample_review)
sequence = tokenizer.texts_to_sequences([cleaned])
padded = pad_sequences(sequence, maxlen=200, padding='post', truncating='post')

# Predict
prediction = model.predict(padded)
sentiment = "Positive" if prediction[0][0] >= 0.5 else "Negative"
print(f"Predicted Sentiment: {sentiment} (Confidence: {prediction[0][0]:.4f})")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
Predicted Sentiment: Positive (Confidence: 0.9694)


### Inference Result

We performed a real-time inference using our trained sentiment classification model. The input was a manually written movie review:

> "**I really enjoyed the movie. The plot was strong and the acting was superb!**"

The review underwent the same preprocessing steps as the training data — including lowercasing, HTML tag and special character removal, and stopword filtering.

After tokenization and padding, the model predicted the sentiment as:

- **Predicted Sentiment:** **Positive**  
- **Confidence Score:** **0.9694**

This high confidence score indicates the model's strong certainty that the review expresses positive sentiment, showcasing its ability to generalize well to unseen data.


Now that the model sucessfully identified positive sentiment next let's try with negative one!

In [10]:
# Example text
sample_review = "The movie was painfully slow and the plot made no sense at all."

# Clean, tokenize, and pad
cleaned = clean_text(sample_review)
sequence = tokenizer.texts_to_sequences([cleaned])
padded = pad_sequences(sequence, maxlen=200, padding='post', truncating='post')

# Predict
prediction = model.predict(padded)
sentiment = "Positive" if prediction[0][0] >= 0.5 else "Negative"
print(f"Predicted Sentiment: {sentiment} (Confidence: {prediction[0][0]:.4f})")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Predicted Sentiment: Positive (Confidence: 0.6287)


### Inference Result (Misclassified Example)

We tested the model using the following clearly negative statement:

> *"The movie was painfully slow and the plot made no sense at all."*

Despite the review expressing clear dissatisfaction, the model predicted:

- **Predicted Sentiment:** **Positive**
- **Confidence Score:** **0.6287**

This misclassification highlights one of the model's limitations — it may struggle with certain types of criticism that lack strong negative keywords or are expressed in more formal or indirect language. This insight could guide future model improvements, such as enhancing context understanding or fine-tuning on a more nuanced dataset.


It fail when predicting negative sentiment this is may be caused the lack of strong negative word so the model only identify pain word as a general sentiment.